In [17]:
import copy
from collections import deque

# States and adjacency list (India map)
states = [
    "AP","AR","AS","BR","CG","GA","GJ","HR","HP","JH","KA","KL","MP","MH",
    "MN","ML","MZ","NL","OD","PB","RJ","SK","TN","TS","TR","UP","UK","WB","JK"
]

adjacency = {
    "AP":["TS","OD","TN","KA"],
    "AR":["AS","NL"],
    "AS":["AR","NL","ML","TR","MZ","MN","WB"],
    "BR":["UP","JH","WB"],
    "CG":["UP","JH","OD","MH"],
    "GA":["MH","KA"],
    "GJ":["MH","RJ"],
    "HR":["PB","HP","UK","RJ"],
    "HP":["JK","PB","HR","UK"],
    "JH":["BR","UP","CG","OD","WB"],
    "KA":["MH","AP","TS","TN","KL","GA"],
    "KL":["KA","TN"],
    "MP":["RJ","UP","CG","MH","GJ"],
    "MH":["GJ","MP","CG","TS","KA","GA"],
    "MN":["AS","MZ","NL"],
    "ML":["AS","TR"],
    "MZ":["AS","MN","TR"],
    "NL":["AR","AS","MN"],
    "OD":["WB","JH","CG","AP","TS"],
    "PB":["JK","HP","HR","RJ"],
    "RJ":["PB","HR","MP","GJ","UP"],
    "SK":["WB"],
    "TN":["AP","KA","KL"],
    "TS":["MH","KA","AP","OD"],
    "TR":["AS","ML","MZ"],
    "UP":["UK","HR","RJ","MP","CG","JH","BR"],
    "UK":["HP","HR","UP"],
    "WB":["BR","JH","OD","AS","SK"],
    "JK":["PB","HP"]
}

colors = ["Red","Green","Blue","Yellow"]

# ---------------------------------------------------------
# Utility functions
# ---------------------------------------------------------
def is_valid(state, assignment, color):
    for neighbor in adjacency[state]:
        if neighbor in assignment and assignment[neighbor] == color:
            return False
    return True

# MRV heuristic: pick variable with fewest legal values
def select_unassigned_variable(assignment, domains):
    unassigned = [v for v in states if v not in assignment]
    return min(unassigned, key=lambda var: len(domains[var]))

# LCV heuristic: order values that constrain neighbors least
def order_domain_values(var, assignment, domains):
    counts = []
    for val in domains[var]:
        count = 0
        for neighbor in adjacency[var]:
            if neighbor not in assignment and val in domains[neighbor]:
                count += 1
        counts.append((val, count))
    # Sort by least constraining
    return [val for val, _ in sorted(counts, key=lambda x: x[1])]

# AC-3 algorithm
def ac3(domains):
    queue = deque([(Xi, Xj) for Xi in states for Xj in adjacency[Xi]])
    while queue:
        Xi, Xj = queue.popleft()
        if revise(domains, Xi, Xj):
            if not domains[Xi]:
                return False
            for Xk in adjacency[Xi]:
                if Xk != Xj:
                    queue.append((Xk, Xi))
    return True

def revise(domains, Xi, Xj):
    revised = False
    to_remove = []
    for x in domains[Xi]:
        if all(x == y for y in domains[Xj]):  # No valid value for neighbor
            to_remove.append(x)
    for val in to_remove:
        domains[Xi].remove(val)
        revised = True
    return revised

# ---------------------------------------------------------
# Solvers
# ---------------------------------------------------------
def backtracking_plain(assignment, steps):
    if len(assignment) == len(states):
        return assignment, steps
    var = [s for s in states if s not in assignment][0]
    for color in colors:
        steps[0] += 1
        if is_valid(var, assignment, color):
            assignment[var] = color
            result, steps = backtracking_plain(assignment, steps)
            if result:
                return result, steps
            assignment.pop(var)
    return None, steps

def backtracking_heuristic(assignment, domains, steps, use_ac3=False):
    if len(assignment) == len(states):
        return assignment, steps
    var = select_unassigned_variable(assignment, domains)
    order = order_domain_values(var, assignment, domains)
    print(f"MRV selected: {var}, LCV order: {order}")
    for color in order:
        steps[0] += 1
        if is_valid(var, assignment, color):
            assignment[var] = color
            saved_domains = copy.deepcopy(domains)
            domains[var] = [color]
            if use_ac3:
                if not ac3(domains):
                    assignment.pop(var)
                    domains = saved_domains
                    continue
            result, steps = backtracking_heuristic(assignment, domains, steps, use_ac3)
            if result:
                return result, steps
            assignment.pop(var)
            domains = saved_domains
    return None, steps

# ---------------------------------------------------------
# Main Execution
# ---------------------------------------------------------
if __name__ == "__main__":
    # Plain backtracking
    result1, steps1 = backtracking_plain({}, [0])
    print("\nPlain Backtracking Result:", result1)
    print("Steps:", steps1[0])

    # Backtracking + MRV + LCV
    domains = {s: colors[:] for s in states}
    result2, steps2 = backtracking_heuristic({}, domains, [0], use_ac3=False)
    print("\nBacktracking + MRV + LCV Result:", result2)
    print("Steps:", steps2[0])

    # Backtracking + MRV + LCV + AC-3
    domains = {s: colors[:] for s in states}
    result3, steps3 = backtracking_heuristic({}, domains, [0], use_ac3=True)
    print("\nBacktracking + MRV + LCV + AC-3 Result:", result3)
    print("Steps:", steps3[0])

    # Comparison table
    print("\nComparison Table:")
    print(f"{'Algorithm':40} {'Steps'}")
    print(f"{'Plain Backtracking':40} {steps1[0]}")
    print(f"{'Backtracking + MRV + LCV':40} {steps2[0]}")
    print(f"{'Backtracking + MRV + LCV + AC-3':40} {steps3[0]}")



Plain Backtracking Result: {'AP': 'Red', 'AR': 'Red', 'AS': 'Green', 'BR': 'Red', 'CG': 'Red', 'GA': 'Red', 'GJ': 'Red', 'HR': 'Red', 'HP': 'Green', 'JH': 'Green', 'KA': 'Green', 'KL': 'Red', 'MP': 'Green', 'MH': 'Blue', 'MN': 'Red', 'ML': 'Red', 'MZ': 'Blue', 'NL': 'Blue', 'OD': 'Blue', 'PB': 'Blue', 'RJ': 'Yellow', 'SK': 'Red', 'TN': 'Blue', 'TS': 'Yellow', 'TR': 'Yellow', 'UP': 'Blue', 'UK': 'Yellow', 'WB': 'Yellow', 'JK': 'Red'}
Steps: 63
MRV selected: AP, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: AR, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: AS, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: BR, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: CG, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: GA, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: GJ, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: HR, LCV order: ['Red', 'Green', 'Blue', 'Yellow']
MRV selected: HP, LCV order: ['Red', 'Gr